In [1]:
import json
import pandas as pd
import numpy as np


In [2]:
file_dir = 'C://Users/azinc/Github/Movies-ETL/'

filename = 'wikipedia-movies.json'


In [3]:
with open(f'{file_dir}{filename}', mode='r') as file:
    wiki_movies_raw = json.load(file)
    
len(wiki_movies_raw)

7311

In [4]:
wiki_movies_raw[7200:7250]

[{'url': 'https://en.wikipedia.org/wiki/Puzzle_(2018_film)',
  'year': 2018,
  'imdb_link': 'https://www.imdb.com/title/tt6933454/',
  'title': 'Puzzle',
  'Directed by': 'Marc Turtletaub',
  'Produced by': ['Wren Arthur',
   'Guy Stodel',
   'Marc Turtletaub',
   'Peter Saraf'],
  'Written by': ['Oren Moverman', 'Polly Mann'],
  'Based on': ['Rompecabezas', 'by', 'Natalia Smirnoff'],
  'Starring': ['Kelly Macdonald',
   'Irrfan Khan',
   'David Denman',
   'Bubba Weiler',
   'Austin Abrams',
   'Liv Hewson'],
  'Music by': "Dustin O'Halloran",
  'Cinematography': 'Chris Norr',
  'Edited by': 'Catherine Haight',
  'Productioncompanies ': ['Big Beach', 'Olive Productions'],
  'Distributed by': 'Sony Pictures Classics',
  'Release date': ['January 23, 2018',
   '(',
   '2018-01-23',
   ')',
   '(',
   'Sundance',
   ')',
   'July 27, 2018',
   '(',
   '2018-07-27',
   ')',
   '(United States)'],
  'Running time': '103 minutes',
  'Country': 'United States',
  'Language': 'English',
  'Bo

In [6]:
kaggle_metadata_df = pd.read_csv(f'{file_dir}movies_metadata.csv',low_memory=False)

ratings_df = pd.read_csv(f'{file_dir}ratings.csv')

In [10]:
kaggle_metadata_df.sample(n=10)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
14069,False,"{'id': 223564, 'name': 'The Cell Collection', ...",0,"[{'id': 27, 'name': 'Horror'}, {'id': 878, 'na...",NaN,18759,tt1185837,en,The Cell 2,The Cusp is a serial killer who kills his vict...,...,2009-02-27,0.0,94.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,In the mind of a mad man,The Cell 2,False,3.4,24.0
19443,False,NaN,0,"[{'id': 35, 'name': 'Comedy'}]",NaN,106124,tt0059581,cs,Perličky na dně,"A manifesto of sorts for the Czech New Wave, t...",...,1966-01-06,0.0,105.0,"[{'iso_639_1': 'cs', 'name': 'Český'}]",Released,NaN,Pearls of the Deep,False,6.2,5.0
19898,False,NaN,0,"[{'id': 35, 'name': 'Comedy'}, {'id': 28, 'nam...",NaN,70863,tt1693843,en,Poolboy - Drowning Out the Fury,In this unearthed lost movie from 1990 that th...,...,2011-07-22,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,No Lifeguard on Duty,Poolboy - Drowning Out the Fury,False,4.3,16.0
29661,False,NaN,0,"[{'id': 9648, 'name': 'Mystery'}, {'id': 80, '...",NaN,3699,tt0067696,en,Sette orchidee macchiate di rosso,"A woman, a survivor of a failed murder attempt...",...,1972-06-30,0.0,92.0,"[{'iso_639_1': 'de', 'name': 'Deutsch'}, {'iso...",Released,NaN,Seven Blood-Stained Orchids,False,6.3,10.0
32936,False,"{'id': 134633, 'name': 'Snow Collection', 'pos...",0,"[{'id': 10751, 'name': 'Family'}, {'id': 10770...",NaN,51836,tt1334278,en,Snow 2: Brain Freeze,"It's three days before Christmas, and Nick Sno...",...,2008-08-08,0.0,86.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Snow 2: Brain Freeze,False,4.4,7.0
15864,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",NaN,342218,tt0080185,sv,Den Åttonde dagen,Children as real human individuals with the di...,...,1979-01-13,0.0,96.0,"[{'iso_639_1': 'sv', 'name': 'svenska'}]",Released,A Young Swedish Movie Event -The First Love,The Eighth Day,False,0.0,0.0
2790,False,NaN,64000000,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",NaN,12618,tt0156934,en,Random Hearts,After the death of their loved ones in a tragi...,...,1999-10-08,74608570.0,133.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,In a perfect world...they never would have met.,Random Hearts,False,5.3,63.0
21848,False,NaN,25000000,"[{'id': 53, 'name': 'Thriller'}, {'id': 80, 'n...",NaN,109091,tt2193215,en,The Counselor,A rich and successful lawyer named Counselor i...,...,2013-10-25,71009334.0,117.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Sin Is A Choice.,The Counselor,False,5.0,724.0
23709,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 36, 'name...",NaN,109716,tt0032396,en,A Dispatch from Reuter's,German Julius Reuter (Edward G. Robinson) send...,...,1940-10-19,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,A Dispatch from Reuter's,False,3.5,3.0
30630,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 10769, '...",NaN,108665,tt0201470,cn,Bui bun si mun,Gao Jian is a lazy and arrogant student of the...,...,1980-05-23,0.0,92.0,"[{'iso_639_1': 'zh', 'name': '普通话'}, {'iso_639...",Released,NaN,The Master,False,5.8,2.0


In [11]:
wiki_movies_df = pd.DataFrame(wiki_movies_raw)

wiki_movies_df.head()

,url,year,imdb_link,title,Directed by,Produced by,Screenplay by,Story by,Based on,Starring,...,Predecessor,Founders,Area served,Products,Services,Russian,Hebrew,Revenue,Operating income,Polish
0,https://en.wikipedia.org/wiki/The_Adventures_o...,1990.0,https://www.imdb.com/title/tt0098987/,The Adventures of Ford Fairlane,Renny Harlin,"[Steve Perry, Joel Silver]","[David Arnott, James Cappe, Daniel Waters]","[David Arnott, James Cappe]","[Characters, by Rex Weiner]","[Andrew Dice Clay, Wayne Newton, Priscilla Pre...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"https://en.wikipedia.org/wiki/After_Dark,_My_S...",1990.0,https://www.imdb.com/title/tt0098994/,"After Dark, My Sweet",James Foley,"[Ric Kidney, Robert Redlin]","[James Foley, Robert Redlin]",NaN,"[the novel, After Dark, My Sweet, by, Jim Thom...","[Jason Patric, Rachel Ward, Bruce Dern, George...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://en.wikipedia.org/wiki/Air_America_(film),1990.0,https://www.imdb.com/title/tt0099005/,Air America,Roger Spottiswoode,Daniel Melnick,"[John Eskow, Richard Rush]",NaN,"[Air America, by, Christopher Robbins]","[Mel Gibson, Robert Downey Jr., Nancy Travis, ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://en.wikipedia.org/wiki/Alice_(1990_film),1990.0,https://www.imdb.com/title/tt0099012/,Alice,Woody Allen,Robert Greenhut,NaN,NaN,NaN,"[Alec Baldwin, Blythe Danner, Judy Davis, Mia ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://en.wikipedia.org/wiki/Almost_an_Angel,1990.0,https://www.imdb.com/title/tt0099018/,Almost an Angel,John Cornell,John Cornell,NaN,NaN,NaN,"[Paul Hogan, Elias Koteas, Linda Kozlowski]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
wiki_movies_df.columns.to_list()

['url',
 'year',
 'imdb_link',
 'title',
 'Directed by',
 'Produced by',
 'Screenplay by',
 'Story by',
 'Based on',
 'Starring',
 'Narrated by',
 'Music by',
 'Cinematography',
 'Edited by',
 'Productioncompany ',
 'Distributed by',
 'Release date',
 'Running time',
 'Country',
 'Language',
 'Budget',
 'Box office',
 'Written by',
 'Genre',
 'Theme music composer',
 'Country of origin',
 'Original language(s)',
 'Producer(s)',
 'Editor(s)',
 'Production company(s)',
 'Original network',
 'Original release',
 'Productioncompanies ',
 'Executive producer(s)',
 'Production location(s)',
 'Distributor',
 'Picture format',
 'Audio format',
 'Voices of',
 'Followed by',
 'Composer(s)',
 'Created by',
 'Also known as',
 'Opening theme',
 'No. of episodes',
 'Preceded by',
 'Author',
 'Publisher',
 'Publication date',
 'Media type',
 'Pages',
 'ISBN',
 'OCLC',
 'LC Class',
 'Cover artist',
 'Series',
 'Set in',
 'Adaptation by',
 'Suggested by',
 'Biographical data',
 'Born',
 'Died',
 'Resti

In [17]:
# Use list comprehension to filter wikipedia json columns to clean up the movie data

wiki_movies = [movie for movie in wiki_movies_raw 
              if ('Director' in movie or 'Directed by' in movie
                  and 'imdb_link' in movie
                 and 'No. of episodes')]
                  
len(wiki_movies)

7080

In [20]:
wiki_movy_df = pd.DataFrame(wiki_movies)

sorted(wiki_movy_df.columns.to_list())

['Adaptation by',
 'Also known as',
 'Animation by',
 'Arabic',
 'Audio format',
 'Based on',
 'Box office',
 'Budget',
 'Camera setup',
 'Cantonese',
 'Chinese',
 'Cinematography',
 'Color process',
 'Composer(s)',
 'Country',
 'Country of origin',
 'Created by',
 'Directed by',
 'Director',
 'Distributed by',
 'Distributor',
 'Edited by',
 'Editor(s)',
 'Executive producer(s)',
 'Followed by',
 'French',
 'Genre',
 'Hangul',
 'Hebrew',
 'Hepburn',
 'Japanese',
 'Label',
 'Language',
 'Length',
 'Literally',
 'Mandarin',
 'McCune–Reischauer',
 'Music by',
 'Narrated by',
 'No. of episodes',
 'Opening theme',
 'Original language(s)',
 'Original network',
 'Original release',
 'Original title',
 'Picture format',
 'Polish',
 'Preceded by',
 'Produced by',
 'Producer',
 'Producer(s)',
 'Production company(s)',
 'Production location(s)',
 'Productioncompanies ',
 'Productioncompany ',
 'Recorded',
 'Release date',
 'Released',
 'Revised Romanization',
 'Romanized',
 'Running time',
 'Russ

In [ ]:
def clean_movie(movie):
    movie = dict(movie) # non-destructive copy of the movie-data
    
    return(movie)

In [21]:
wiki_movies_df[wiki_movies_df['Adaptation by'].notnull()]

,url,year,imdb_link,title,Directed by,Produced by,Screenplay by,Story by,Based on,Starring,...,Predecessor,Founders,Area served,Products,Services,Russian,Hebrew,Revenue,Operating income,Polish
548,https://en.wikipedia.org/wiki/The_Last_of_the_...,1992.0,https://www.imdb.com/title/tt0104691/,The Last of the Mohicans,Michael Mann,"[Michael Mann, Hunt Lowry]","[Michael Mann, Christopher Crowe]",NaN,"[The Last of the Mohicans, by, James Fenimore ...","[Daniel Day-Lewis, Madeleine Stowe, Jodhi May]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
818,https://en.wikipedia.org/wiki/The_Nightmare_Be...,1993.0,https://www.imdb.com/title/tt0107688/,The Nightmare Before Christmas,Henry Selick,"[Tim Burton, Denise Di Novi]",Caroline Thompson,NaN,"[Story and characters, by Tim Burton]","[Danny Elfman, Chris Sarandon, Catherine O'Har...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
